In [1]:
!pip install xgboost
!apt-get -y update
!apt-get -y install libatlas-base-dev

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease              
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease              
Reading package lists... Done                     
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-5).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.


In [2]:
# !conda install -y -c conda-forge py-xgboost

In [3]:
# !conda install -y -c conda-forge py-xgboost=*=cuda*

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from xgboost import XGBClassifier
import xgboost as xgb

In [5]:
print(xgb.__version__)

1.6.2


In [6]:
df = pd.read_csv("../llamatales/story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [7]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [8]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10
# NUM_PROMPTS = 2

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'../llamatales/hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
            # print(curr_hidden_states.shape)
            
            #By layer
            for layer in range(1):
                curr_layer_hidden_states = curr_hidden_states[layer][0].astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
                # print(curr_layer_hidden_states.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(curr_layer_hidden_states)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [curr_layer_hidden_states]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:12<00:00, 13.75it/s]


In [9]:
layer_hs_array = hidden_states_by_layer["layer_0"]
print(layer_hs_array[0].shape)
print(layer_hs_array[1].shape)

(270, 512)
(349, 512)


In [10]:
min_story_len = min(df["len_generated_story"])
min_story_len

21

In [11]:
max_story_len = max(df[:NUM_PROMPTS*1000]["len_generated_story"])

In [12]:
max_new_story_len = max(df["len_new_story"])

In [13]:
max_new_story_len = max(df[:NUM_PROMPTS*1000]["len_new_story"])

In [14]:
max_new_story_len

523

In [15]:
df[df["len_generated_story"] >= max_story_len].shape

(13, 6)

In [16]:
df[df["len_new_story"] >= max_new_story_len]

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
2360,3,Once upon a time there were two children,"Once upon a time there were two children, Emma...",./hidden_states/prompt_3.npz,522,523
2628,3,Once upon a time there were two children,Once upon a time there were two children named...,./hidden_states/prompt_3.npz,522,523
7795,8,Once upon a time there was a wolf,Once upon a time there was a wolf named Max. M...,./hidden_states/prompt_8.npz,522,523
9178,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,522,523
9201,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,522,523
9270,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,522,523
9313,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,522,523
9603,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,522,523
9626,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,522,523
9685,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,522,523


In [17]:
max(df[:NUM_PROMPTS*1000]["len_new_story"])

523

In [18]:
hs_shapes = []
for i in range(len(layer_hs_array)):
    hs_shapes.append(layer_hs_array[i].shape[0])

In [19]:
np.array(hs_shapes)[9685]

522

In [20]:
import numpy as np

def safe_split(total, train_ratio=0.8):
    train = int(train_ratio * total)   # floor
    test  = total - train              # leftover
    return train, test

def build_dataset(curr_context_level_hs, curr_labels, train_ratio=0.8):
    """
    Groups samples by prompt_id, splits safely into train/test,
    and returns X_train, y_train, X_test, y_test.
    """

    unique_ids = sorted(set(curr_labels))

    X_train_list = []
    X_test_list = []
    y_train_list = []
    y_test_list = []

    for pid in unique_ids:

        # get samples for this prompt id
        mask = (curr_labels == pid)
        X_pid = curr_context_level_hs[mask]
        y_pid = curr_labels[mask]

        total = len(X_pid)
        train_n, test_n = safe_split(total, train_ratio)

        # split
        X_train_list.append(X_pid[:train_n])
        y_train_list.append(y_pid[:train_n])

        X_test_list.append(X_pid[train_n:])
        y_test_list.append(y_pid[train_n:])

    # concatenate all prompt-id blocks
    X_train = np.concatenate(X_train_list, axis=0)
    y_train = np.concatenate(y_train_list, axis=0)
    X_test  = np.concatenate(X_test_list, axis=0)
    y_test  = np.concatenate(y_test_list, axis=0)

    return X_train, y_train, X_test, y_test

In [ ]:
#Context-level analysis
# min_story_len = min(df["len_new_story"])
max_story_len = max(df[:NUM_PROMPTS*1000]["len_new_story"]) #save hidden states when generating new story - length of hidden states is of the old story (522)
for context_level in range(11, max_story_len):
    curr_context_level_hs = []
    curr_labels = []
    
    curr_hs_shapes = []
    for i in range(len(layer_hs_array)):
        # print(np.array(layer_hs_array[i][:context_level]).shape) #see if taking context level more than available causes error?
        layer_hs_upto_context = layer_hs_array[i][10:context_level]
        
        padded_layer_hs_upto_context = np.zeros((context_level, layer_hs_upto_context.shape[1]), dtype = np.float32)
        padded_layer_hs_upto_context[:layer_hs_upto_context.shape[0]] = layer_hs_upto_context
        curr_hs_shapes.append(layer_hs_array[i].shape[0])
        curr_context_level_hs.append(np.array(padded_layer_hs_upto_context).flatten())
        curr_labels.append(df.iloc[i].prompt_id)
    
    curr_context_level_hs = np.array(curr_context_level_hs).astype('float32')
    # print(curr_context_level_hs.shape) #print shape without samples less than context level
    curr_labels = np.array(curr_labels) - 1
    
    if np.var(curr_context_level_hs) < 1e-6:
        continue
    
    unique_ids = sorted(set(curr_labels))
    
    X_train, y_train, X_test, y_test = build_dataset(curr_context_level_hs, curr_labels)
    
    # classifier = XGBClassifier(seed = 42, objective = 'multi:softmax', eval_metric = "merror", num_class = len(unique_ids))
    classifier = XGBClassifier(seed = 42, objective = 'multi:softmax', eval_metric = "merror", num_class = len(unique_ids), tree_method='gpu_hist', predictor='gpu_predictor', device='cuda')
    classifier.fit(X_train, y_train)
    preds = classifier.predict(X_test)
    
    print("Train Data Shape: ", X_train.shape)
    print("Train Labels Shape: ", y_train.shape)
    print("Test Data Shape: ", X_test.shape)
    print("Test Labels Shape: ", y_test.shape)
    
    print("Predictions Shape: ", preds.shape)
    
    print("Unique Preds: ", pd.Series(preds).value_counts())
    print("Unique Test Labels: ", pd.Series(y_test).value_counts())
    print(f"Accuracy: {np.mean(preds == y_test)}")
    
    # print(max(curr_hs_shapes))

[16:40:32] WARNING: ../src/learner.cc:627: 
Parameters: { "device" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Train Data Shape:  (8000, 5632)
Train Labels Shape:  (8000,)
Test Data Shape:  (2000, 5632)
Test Labels Shape:  (2000,)
Predictions Shape:  (2000,)
Unique Preds:  0    355
3    268
2    261
6    213
8    210
9    210
4    188
1    134
7    128
5     33
dtype: int64
Unique Test Labels:  0    200
1    200
2    200
3    200
4    200
5    200
6    200
7    200
8    200
9    200
dtype: int64
Accuracy: 0.689
[16:40:53] WARNING: ../src/learner.cc:627: 
Parameters: { "device" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or so